In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.cm as cm
import requests
#import dill
from bs4 import BeautifulSoup
#from datetime import datetime
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import XML, fromstring, tostring
# from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime, timedelta
import time
import os
import pandas
from ast import literal_eval
import os


In [2]:
my_path = os.path.abspath("cycleTrackImages_Bengaluru_2/")

def getTime():
    now = datetime.now()
    current_time = now.strftime("%d%b%Y_%H-%M")
    return current_time


The save data function saves all the data returned from the request to local file

In [3]:
def save_data():
    page = requests.get('https://traffic.api.here.com/traffic/6.2/flow.xml?app_id=wvzQzqmPlU1T9tjf0YLU&app_code=b_is4SmSRfh8e0-Mr2-low&bbox=12.907945, 77.512298;13.014777, 77.693619&responseattributes=sh,fc&units=metric')
    soup = BeautifulSoup(page.text, "lxml")
    roads = soup.find_all('fi')
    a1=[]
    loc_list_hv=[]
    lats=[]
    longs=[]
    sus=[]
    ffs=[]
    des=[]
    les=[]
    pcs=[]
    cns=[]
    jfs=[]
    sps=[]
    cns=[] 
    c=0
    for road in roads:
        #for j in range(0,len(shps)):
        myxml = fromstring(str(road))
        fc=5
        for child in myxml:
            #print(child.tag, child.attrib)
            if('fc' in child.attrib):
                fc=int(child.attrib['fc'])
            if('cn' in child.attrib):
                cn=float(child.attrib['cn'])
            if('su' in child.attrib):
                su=float(child.attrib['su'])
            if('ff' in child.attrib):
                ff=float(child.attrib['ff'])
            if('de' in child.attrib):
                de=child.attrib['de']
            if('le' in child.attrib):
                le=float(child.attrib['le'])
            if('pc' in child.attrib):
                pc=float(child.attrib['pc'])
            if('jf' in child.attrib):
                jf=float(child.attrib['jf'])
            if('sp' in child.attrib):
                sp=float(child.attrib['sp'])
        if((fc<=3)):
            shps=road.find_all("shp")
            for j in range(0,len(shps)):
                latlong=shps[j].text.replace(',',' ').split()
                #loc_list=[]
                la=[]
                lo=[]
                su1=[]
                ff1=[] 
                for i in range(0,int(len(latlong)/2)):
                    loc_list_hv.append([de,float(latlong[2*i]),float(latlong[2*i+1]),float(su),float(ff),float(le),float(pc),float(cn),float(jf),float(sp),float(cn),getTime()])
                    la.append(float(latlong[2*i]))
                    lo.append(float(latlong[2*i+1]))
                    su1.append(float(su))
                    ff1.append(float(ff)) 
                lats.append(la)
                longs.append(lo)
                sus.append(np.mean(su1))
                ffs.append(np.mean(ff1))
                des.append(de)
                les.append(le)
                pcs.append(pc)
                jfs.append(jf)
                sps.append(sp)
                cns.append(cn)
    df = pandas.DataFrame(list(zip(lats, longs,sus,ffs,des,les,pcs,jfs,sps,cns)),columns =['Lat', 'Long','SU','FF',"Name","Les","Pcs","Jfs","Sps","CNs"]) 
    df.to_csv(os.path.join(my_path,'speedProfileData_'+getTime()+'.csv'), sep=',',index=False)


The following code runs the save data function every 15 minutes

In [ ]:
# import sys
# !{sys.executable} -m pip install schedule
import schedule
import time
schedule.every(.15).minutes.do(save_data)
while True:
    schedule.run_pending()
    time.sleep(1)